In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv('heart_disease_dataset.csv',delimiter=';')
df=data.copy()
df.head()

,age,sex,chest_pain_type,resting_blood_pressure,cholesterol,fasting_blood_sugar,rest_ecg,max_heart_rate_achieved,exercise_induced_angina,st_depression,st_slope,num_major_vessels,thalassemia,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      303 non-null    int64  
 1   sex                      303 non-null    int64  
 2   chest_pain_type          303 non-null    int64  
 3   resting_blood_pressure   303 non-null    int64  
 4   cholesterol              303 non-null    int64  
 5   fasting_blood_sugar      303 non-null    int64  
 6   rest_ecg                 303 non-null    int64  
 7   max_heart_rate_achieved  303 non-null    int64  
 8   exercise_induced_angina  303 non-null    int64  
 9   st_depression            303 non-null    float64
 10  st_slope                 303 non-null    int64  
 11  num_major_vessels        303 non-null    int64  
 12  thalassemia              303 non-null    int64  
 13  target                   303 non-null    int64  
dtypes: float64(1), int64(13)
m

In [4]:
X=df.drop(columns=['target'],axis=1)
y=df.target.values.reshape(-1,1)

In [5]:
from sklearn.preprocessing import RobustScaler

In [6]:
rc=RobustScaler()
X=rc.fit_transform(X)

In [7]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.2,stratify=y ,random_state=42)

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset,DataLoader

In [11]:
X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()
y_train = torch.from_numpy(y_train).float()
y_test = torch.from_numpy(y_test).float()

In [12]:
train_dataset = TensorDataset(X_train,y_train)
test_dataset = TensorDataset(X_test,y_test)

In [13]:
train_loader = DataLoader(train_dataset,batch_size=500,shuffle=True)
test_loader = DataLoader(test_dataset,shuffle=True)

In [15]:
df.shape

(303, 14)

In [16]:
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        
        self.fc1 = nn.Linear(13,50)
        self.out = nn.Linear(50,1)

    def forward(self,X):
        X = F.relu(self.fc1(X))
        X = self.out(X)
        return X
    

In [17]:
model = Model()

In [18]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [19]:
get_n_params(model)

751

In [20]:
#defining paramtres
num_epochs = 30
lr_rate = 0.3
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(),lr=lr_rate)

In [21]:
# creating a supporting function
def binary_acc(y_pred,y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    
    correct_results_sum = (y_pred_tag==y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc*100)
    return acc

In [22]:
model.train()
for e in range(1,num_epochs+1):
    epoch_loss = 0
    epoch_acc = 0 
    for X_batch,y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        
        loss = criterion(y_pred.flatten(),y_batch.flatten().float())
        acc = binary_acc(y_pred.flatten(),y_batch.flatten().float())
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    print(f'Epoch: {e+0:03}: |Loss:{epoch_loss/len(train_loader):.5f}|Acc: {epoch_acc/len(train_loader):.3f}')

Epoch: 001: |Loss:0.68855|Acc: 56.000
Epoch: 002: |Loss:0.66715|Acc: 67.000
Epoch: 003: |Loss:0.64842|Acc: 70.000
Epoch: 004: |Loss:0.63139|Acc: 73.000
Epoch: 005: |Loss:0.61546|Acc: 75.000
Epoch: 006: |Loss:0.60031|Acc: 77.000
Epoch: 007: |Loss:0.58573|Acc: 77.000
Epoch: 008: |Loss:0.57168|Acc: 77.000
Epoch: 009: |Loss:0.55807|Acc: 77.000
Epoch: 010: |Loss:0.54500|Acc: 78.000
Epoch: 011: |Loss:0.53241|Acc: 79.000
Epoch: 012: |Loss:0.52041|Acc: 79.000
Epoch: 013: |Loss:0.50892|Acc: 80.000
Epoch: 014: |Loss:0.49794|Acc: 80.000
Epoch: 015: |Loss:0.48758|Acc: 80.000
Epoch: 016: |Loss:0.47790|Acc: 81.000
Epoch: 017: |Loss:0.46874|Acc: 81.000
Epoch: 018: |Loss:0.46014|Acc: 81.000
Epoch: 019: |Loss:0.45216|Acc: 81.000
Epoch: 020: |Loss:0.44470|Acc: 81.000
Epoch: 021: |Loss:0.43777|Acc: 81.000
Epoch: 022: |Loss:0.43133|Acc: 81.000
Epoch: 023: |Loss:0.42532|Acc: 81.000
Epoch: 024: |Loss:0.41969|Acc: 82.000
Epoch: 025: |Loss:0.41443|Acc: 82.000
Epoch: 026: |Loss:0.40951|Acc: 84.000
Epoch: 027: 

In [23]:
y_pred_list = []
model.eval()
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        torch.no_grad()
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.numpy())

y_pred_list = [a.squeeze().tolist() for  a in  y_pred_list]

In [24]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [25]:
print(confusion_matrix(y_test,y_pred_list))

[[11 17]
 [12 21]]


In [26]:
print(classification_report(y_test,y_pred_list))

              precision    recall  f1-score   support

         0.0       0.48      0.39      0.43        28
         1.0       0.55      0.64      0.59        33

    accuracy                           0.52        61
   macro avg       0.52      0.51      0.51        61
weighted avg       0.52      0.52      0.52        61

